# Tutorial: Events in data classes

**Author:** Masoud Abedinifar & Julius Welzel

**Last update:** Thu 14 Mar 2024

## Learning objectives
By the end of this tutorial:

- Load data from a recording that belongs to one of the available datasets.
- Store events from algorithms in the datclass per recording file.
- Export [events to BIDS format](https://bids-specification.readthedocs.io/en/stable/modality-specific-files/task-events.html).

## Import libraries
The necessary libraries such as numpy, matplotlib.pyplot, dataset (mobilised), Paraschiv-Ionescu gait sequence detection, and Paraschiv-Ionescu initial contact detection algorithms are imported from their corresponding modules. Make sure that you have all the required libraries and modules installed before running this code. You also may need to install the 'kielmat' library and its dependencies if you haven't already.

In [8]:
from kielmat.datasets import mobilised
from kielmat.modules.gsd import ParaschivIonescuGaitSequenceDetection

First load the data and put in the desired dataclasses.

In [9]:
# load the data from mobilised dataset
recording = mobilised.load_recording()

# specify which tracking system you want to use
tracking_sys = 'SU'

In [10]:
# Load lower back acceleration data
acceleration_data = recording.data[tracking_sys][
    ["LowerBack_ACCEL_x", "LowerBack_ACCEL_y", "LowerBack_ACCEL_z"]
]

In [11]:
# Get the corresponding sampling frequency directly from the recording
sampling_frequency = recording.channels[tracking_sys][
    recording.channels[tracking_sys]["name"] == "LowerBack_ACCEL_x"
]["sampling_frequency"].values[0]
print(f"Sampling frequency: {sampling_frequency} Hz")

Sampling frequency: 100.0 Hz


The events are put into a pandas DataFrame, and follow the conventions outlined in the BIDS documentation (i.e. https://bids-specification.readthedocs.io/en/stable/modality-specific-files/task-events.html).

### Gait sequence events in dataclass

In [12]:
# Create an instance of the ParaschivIonescuGaitSequenceDetection class
gsd = ParaschivIonescuGaitSequenceDetection()

# Call the gait sequence detection using gsd.detect to detect gait sequences
gsd = gsd.detect(
    data=acceleration_data, sampling_freq_Hz=sampling_frequency, plot_results=False
)

36 gait sequence(s) detected.


In [13]:
# Add events to the recording as a dictionary including tracking system and events
gait_sequence_events = gsd.gait_sequences_
recording.add_events(tracking_system=tracking_sys, new_events=gait_sequence_events)
print(f"events: {recording.events}")

events: {'SU':        onset  duration     event_type tracking_system
0     22.650    17.075  gait sequence            None
1     49.150     7.475  gait sequence            None
2     97.025   120.400  gait sequence            None
3    229.550     9.225  gait sequence            None
4    247.900    29.075  gait sequence            None
5    296.225   189.600  gait sequence            None
6    490.300    25.575  gait sequence            None
7    562.925    15.075  gait sequence            None
8    581.900    18.875  gait sequence            None
9    607.050    56.600  gait sequence            None
10   667.325   101.900  gait sequence            None
11   784.500    42.775  gait sequence            None
12   835.675   174.675  gait sequence            None
13  1034.900    42.050  gait sequence            None
14  1103.075    39.475  gait sequence            None
15  1153.750    13.125  gait sequence            None
16  1184.900     5.775  gait sequence            None
17  1219.175 

### Store events to events.tsv file following the BIDS convention

Add some information about the recording first which is necessary for the BIDS file name convention.
KielMAT has some implemented check on the information to make sure that the file name is in the correct format.

In [14]:
recording.add_info("Subject", "CHF01")
recording.add_info("Task", "walking_outside")

Please notice that we a not to strict with the user. We just give a warning if the file name is not in BIDS like format. However, the user can still continue with the process.
But you better believe that the BIDS police will come and get you if you don't follow the rules.

Now as we have the events in the dataclass, we can export them to a [BIDS compatible events](https://bids-specification.readthedocs.io/en/stable/modality-specific-files/task-events.html) file.

In [17]:
recording.export_events(file_path = r'../examples/data', file_name='gait_sequence.csv')